In [1]:

%load_ext autoreload
%autoreload 2

In [2]:
import sympy
from sympy.matrices.expressions import HadamardProduct
from sympy import ccode, symbols
from sympy.codegen.ast import real, FunctionPrototype
from sympy.codegen.ast import FunctionDefinition, Return
from sympy.utilities.codegen import codegen
from wgsl_codegen import WgslCodeGen

def blend(a, b):
    return sympy.Matrix([
        a[0] + (1 - a[3]) * b[3] * b[0],
        a[1] + (1 - a[3]) * b[3] * b[1],
        a[2] + (1 - a[3]) * b[3] * b[2],
        a[3] + (1 - a[3]) * b[3],
    ])


def blend_bwd(a,b,inc_grad):
    jacobian_a = blend(a,b).jacobian(a)
    jacobian_b = blend(a,b).jacobian(b)
    return jacobian_a * inc_grad, jacobian_b * inc_grad

x,y = sympy.symbols('x y')

a = sympy.Matrix(
    [
        sympy.Function('r_a')(x,y),
        sympy.Function('g_a')(x,y),
        sympy.Function('b_a')(x,y),
        sympy.Function('\\alpha_a')(x,y)
    ]
)

b = sympy.Matrix(
    [
        sympy.Function('r_b')(x,y),
        sympy.Function('g_b')(x,y),
        sympy.Function('b_b')(x,y),
        sympy.Function('\\alpha_b')(x,y)
    ]
)



import sympy
from wgsl_print import WGSLPrinter
a_v = sympy.MatrixSymbol('a',4,1)
b_v = sympy.MatrixSymbol('b',4,1)
a_dx = sympy.MatrixSymbol('a_dx',4,1)
b_dx = sympy.MatrixSymbol('b_dx',4,1)
a_dy = sympy.MatrixSymbol('a_dy',4,1)
b_dy = sympy.MatrixSymbol('b_dy',4,1)
a_dxy = sympy.MatrixSymbol('a_dxy',4,1)
b_dxy = sympy.MatrixSymbol('b_dxy',4,1)


blend_c = blend(a,b)
blend_dx = blend(a,b).diff(x)
blend_dy = blend(a,b).diff(y)
blend_dxy = blend(a,b).diff(x,y)

subs = {
    a[i]: a_v[i] for i in range(len(a))} | \
    {b[i]: b_v[i] for i in range(len(b))} | \
    {a[i].diff(x): a_dx[i] for i in range(len(a))} | \
    {a[i].diff(y): a_dy[i] for i in range(len(a))} | \
    {b[i].diff(x): b_dx[i] for i in range(len(b))} | \
    {b[i].diff(y): b_dy[i] for i in range(len(b))} | \
    {a[i].diff(x,y): a_dxy[i] for i in range(len(a))} | \
    {b[i].diff(x,y): b_dxy[i] for i in range(len(b))}

print(sympy.utilities.codegen.codegen( ('blend_gradient', 
                                        [
                                            sympy.Equality(sympy.MatrixSymbol("color",4,1),blend_c.subs(subs)),
                                            sympy.Equality(sympy.MatrixSymbol("dblend_dx",4,1),blend_dx.subs(subs)),
                                            sympy.Equality(sympy.MatrixSymbol("dblend_dy",4,1),blend_dy.subs(subs)),
                                            sympy.Equality(sympy.MatrixSymbol("dblend_dxy",4,1),blend_dxy.subs(subs)),
                                        ],
                                    ), code_gen=WgslCodeGen(documentation="generated with sympy"), header=False)[0][1])


/// generated with sympy
fn blend_gradient(a: vec4<f32>, a_dx: vec4<f32>, a_dxy: vec4<f32>, a_dy: vec4<f32>, b: vec4<f32>, b_dx: vec4<f32>, b_dxy: vec4<f32>, b_dy: vec4<f32>, color: ptr<function,vec4<f32>>, dblend_dx: ptr<function,vec4<f32>>, dblend_dxy: ptr<function,vec4<f32>>, dblend_dy: ptr<function,vec4<f32>>) {

   (*color)[0] = (1 - a[3])*b[0]*b[3] + a[0];
   (*color)[1] = (1 - a[3])*b[1]*b[3] + a[1];
   (*color)[2] = (1 - a[3])*b[2]*b[3] + a[2];
   (*color)[3] = (1 - a[3])*b[3] + a[3];
   (*dblend_dx)[0] = (1 - a[3])*b[0]*b_dx[3] + (1 - a[3])*b[3]*b_dx[0] + a_dx[0] - a_dx[3]*b[0]*b[3];
   (*dblend_dx)[1] = (1 - a[3])*b[1]*b_dx[3] + (1 - a[3])*b[3]*b_dx[1] + a_dx[1] - a_dx[3]*b[1]*b[3];
   (*dblend_dx)[2] = (1 - a[3])*b[2]*b_dx[3] + (1 - a[3])*b[3]*b_dx[2] + a_dx[2] - a_dx[3]*b[2]*b[3];
   (*dblend_dx)[3] = (1 - a[3])*b_dx[3] - a_dx[3]*b[3] + a_dx[3];
   (*dblend_dxy)[0] = -(a[3] - 1)*b[0]*b_dxy[3] - (a[3] - 1)*b[3]*b_dxy[0] - (a[3] - 1)*b_dx[0]*b_dy[3] - (a[3] - 1)*b_dx[3]*b_dy

In [3]:
expr = blend(a,b)
print(sympy.utilities.codegen.codegen( ('blend', blend(a,b).subs(subs),
                                    ), code_gen=WgslCodeGen(documentation="generated with sympy"), header=False)[0][1])


/// generated with sympy
fn blend(a: vec4<f32>, b: vec4<f32>) -> vec4<f32> {

   var blend_result:vec4<f32>;
   blend_result[0] = (1 - a[3])*b[0]*b[3] + a[0];
   blend_result[1] = (1 - a[3])*b[1]*b[3] + a[1];
   blend_result[2] = (1 - a[3])*b[2]*b[3] + a[2];
   blend_result[3] = (1 - a[3])*b[3] + a[3];
   return blend_result;

}



In [135]:

from wgsl_print import print_wgsl


class fract(sympy.Function):
    @classmethod
    def eval(cls,x):
        # Forward pass: frac(x) = x - floor(x)
        return None
    
    def _eval_derivative(self, s):
        return self.args[0].diff(s)
    


# Example usage
x,y = sympy.symbols('x,y',real=True)
fract(0,x**2).diff(x)

TypeError: fract takes exactly 1 argument (2 given)

In [133]:

class NoDiffAppliedUndef(sympy.core.function.AppliedUndef):

    def _eval_derivative(self, s):
        return sympy.S.Zero
   

class NoDiffFunction(sympy.core.function.UndefinedFunction):
    """
    The (meta)class of undefined functions.
    """
    def __new__(mcl, name, bases=(NoDiffAppliedUndef,), __dict__=None, **kwargs):
        return super(NoDiffFunction, mcl).__new__(mcl, name, bases, __dict__, **kwargs)

x = sympy.symbols('x')
a = NoDiffFunction("test")
a(x).diff(x)

0

In [ ]:


from sympy import HadamardProduct

textureLoad3D = NoDiffFunction('volumeRead3D')
textureLoad1D = NoDiffFunction('volumeRead1D')
textureDimensions3D = NoDiffFunction('textureSize3D')
textureDimensions2D = NoDiffFunction('textureSize2D')

vec3i32 =  sympy.Function('vec3<i32>')
i32 =  sympy.Function('i32')
f32 = sympy.Function('f32')

aabb_min = sympy.MatrixSymbol('settings.volume_aabb.min',3,1)
aabb_max = sympy.MatrixSymbol('settings.volume_aabb.max',3,1)
vmin = sympy.Symbol('settings.vmin')
vmax = sympy.Symbol('settings.vmax')
cmap = sympy.Symbol("cmap")
step_size = sympy.Symbol("settings.step_size")
distance_scale = sympy.Symbol("settings.distance_scale")

def sample_volume(volume,volume_size,p):
    p_n = HadamardProduct(p-aabb_min,(aabb_max - aabb_min).applyfunc(lambda x: 1/x))
    volume_size = sympy.Matrix([
        f32(textureDimensions3D(volume,0)),
        f32(textureDimensions3D(volume,1)),
        f32(textureDimensions3D(volume,2)),
    ])
    x_s, y_s, z_s = [v*s-0.5 for v,s in zip(p_n, volume_size)] 
    x, y, z = fract(x_s), fract(y_s), fract(z_s)
    weights = [
        (1-x)*(1-y)*(1-z),
        (1-x)*(1-y)*z,
        (1-x)*y*(1-z),
        (1-x)*y*z,
        x*(1-y)*(1-z),
        x*(1-y)*z,
        x*y*(1-z),
        x*y*z
    ]
    pos_i = sympy.Matrix([
        sympy.floor(x_s),
        sympy.floor(y_s),
        sympy.floor(z_s)
    ])
    corners = [
        textureLoad3D(volume, vec3i32(pos_i + sympy.Matrix([i//4,(i//2)%2,i%2])),0) for i in range(8)
    ]
        
    return sum(w*c for w,c in zip(weights, corners))

def sample_cmap(cmap, value):
    value_n = (value - vmin) / (vmax - vmin)
    # value_n = sympy.Max(0.0, sympy.Min(1.0, value_n))
    n = f32(textureDimensions2D(cmap, 0))
    pos_n = value_n * n - 0.5
    pos_f = fract(pos_n)
    pos_left = i32(sympy.floor(pos_n))
    pos_right = pos_left + 1
    left_v = sympy.Matrix([
        textureLoad1D(cmap, pos_left,0),
        textureLoad1D(cmap, pos_left,1),
        textureLoad1D(cmap, pos_left,2),
        textureLoad1D(cmap, pos_left,3),  
    ])
    
    right_v = sympy.Matrix([
        textureLoad1D(cmap, pos_right,0),
        textureLoad1D(cmap, pos_right,1),
        textureLoad1D(cmap, pos_right,2),
        textureLoad1D(cmap, pos_right,3),  
    ])
    result = left_v * (1.0 - pos_f) + right_v * pos_f
    return result

def ndc_to_world(ndc,inv_view,inv_proj):
    ndc_h = sympy.Matrix([ndc[0], ndc[1], ndc[2], 1.])
    world_h = inv_view * inv_proj * ndc_h
    # world_h = world_h / world_h[3]
    return sympy.Matrix(world_h[:3])

def rescale_alpha(color):
    a = (1-1/255)*color[3]
    return sympy.Matrix([
        color[0],
        color[1],
        color[2],
        1-sympy.Pow(1-a, step_size*distance_scale)
        a,
    ])

volume = sympy.Symbol("volume")
volume_size = sympy.MatrixSymbol('volume_size',3,1)
p = sympy.MatrixSymbol('p',3,1)
inv_view = sympy.MatrixSymbol('inv_view',4,4)
inv_proj = sympy.MatrixSymbol('inv_proj',4,4)
pos3d = sympy.MatrixSymbol('pos',3,1)

pos2d = sympy.MatrixSymbol("pos_ndc",3,1)
pos3d = ndc_to_world(pos2d,inv_view,inv_proj)
value = sample_volume(volume, volume_size, pos3d)
color = sample_cmap(cmap,value)

color_a = rescale_alpha(color)


print(sympy.utilities.codegen.codegen( [
        ('sample_volume',sample_volume(volume, volume_size, p)),
        ('sample_cmap',sample_cmap(cmap,sympy.Symbol("value"))),
        ('ndc_to_world',ndc_to_world(pos3d,inv_view,inv_proj)),
        ("rescale_alpha",rescale_alpha(sympy.MatrixSymbol("color",4,1))),
        ('color_grad',[
            sympy.Equality(sympy.MatrixSymbol("color_dx",4,1),color_a.diff(pos2d[0]),evaluate=False),
            sympy.Equality(sympy.MatrixSymbol("color_dy",4,1),color_a.diff(pos2d[1]),evaluate=False),
            sympy.Equality(sympy.MatrixSymbol("color_dxy",4,1),color_a.diff(pos2d[0],pos2d[1]),evaluate=False),
            ])
    ],
    global_vars=[volume,aabb_min,aabb_max,vmin,vmax,cmap,distance_scale,step_size],
    code_gen=WgslCodeGen(documentation="generated with sympy",cse=True), header=False)[0][1])



/// generated with sympy
fn sample_volume(p: vec3<f32>) -> f32 {
   let x0 = -settings.volume_aabb.min[2];
   let x1 = (x0 + p[2])*f32(textureSize3D(volume, 2))/(x0 + settings.volume_aabb.max[2]) - 0.5;
   let x2 = fract(x1);
   let x3 = -settings.volume_aabb.min[1];
   let x4 = (x3 + p[1])*f32(textureSize3D(volume, 1))/(x3 + settings.volume_aabb.max[1]) - 0.5;
   let x5 = fract(x4);
   let x6 = -settings.volume_aabb.min[0];
   let x7 = (x6 + p[0])*f32(textureSize3D(volume, 0))/(x6 + settings.volume_aabb.max[0]) - 0.5;
   let x8 = fract(x7);
   let x9 = x5*x8;
   let x10 = floor(x7);
   let x11 = x10 + 1;
   let x12 = floor(x4);
   let x13 = x12 + 1;
   let x14 = floor(x1);
   let x15 = x14 + 1;
   let x16 = 1 - x8;
   let x17 = x16*x5;
   let x18 = 1 - x5;
   let x19 = x18*x8;
   let x20 = 1 - x2;
   let x21 = x16*x18;

   var sample_volume_result:f32;
   sample_volume_result = x17*x2*volumeRead3D(volume, vec3<i32>(vec3<f32>(x10, x13, x15)), 0) + x17*x20*volumeRead3D(volume, vec3<i32


/// generated with sympy
fn color_grad(inv_proj: mat4x4<f32>, inv_view: mat4x4<f32>, pos_ndc: vec3<f32>, color_dx: ptr<function,vec4<f32>>, color_dxy: ptr<function,vec4<f32>>, color_dy: ptr<function,vec4<f32>>) {
   let x0 = -settings.vmin;
   let x1 = f32(textureSize2D(cmap, 0))/(settings.vmax + x0);
   let x2 = f32(textureSize3D(volume, 2));
   let x3 = -settings.volume_aabb.min[2];
   let x4 = 1.0/(x3 + settings.volume_aabb.max[2]);
   let x5 = 1.0*inv_proj[3][0];
   let x6 = 1.0*inv_proj[3][1];
   let x7 = 1.0*inv_proj[3][2];
   let x8 = 1.0*inv_proj[3][3];
   let x9 = inv_proj[0][0]*inv_view[0][2] + inv_proj[0][1]*inv_view[1][2] + inv_proj[0][2]*inv_view[2][2] + inv_proj[0][3]*inv_view[3][2];
   let x10 = inv_proj[1][0]*inv_view[0][2] + inv_proj[1][1]*inv_view[1][2] + inv_proj[1][2]*inv_view[2][2] + inv_proj[1][3]*inv_view[3][2];
   let x11 = x2*x4*(x10*pos_ndc[1] + x3 + x5*inv_view[0][2] + x6*inv_view[1][2] + x7*inv_view[2][2] + x8*inv_view[3][2] + x9*pos_ndc[0] + (inv_proj[2][0

In [121]:
pos2d[0].is_real

In [106]:
print_wgsl(sympy.Min(0,x).diff(x).replace(x,0))

1.0/2.0
